In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import tensorflow as tf
import os
from keras.regularizers import L1L2

# Função personalizada para calcular o MAPE como loss
def mape_loss(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)
    return tf.reduce_mean(tf.abs((y_true - y_pred) / y_true)) * 100

# Função para criar datasets
def create_dataset(data, look_back=1):
    X, Y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back)])
        Y.append(data[i + look_back])
    return np.array(X), np.array(Y)

# Função para construir modelo dinâmico
def build_dynamic_model(input_shape, neurons=100, learning_rate=0.01, n_layers=2):
    model = Sequential()
    
    model.add(LSTM(
        neurons, 
        return_sequences=(n_layers > 1), 
        input_shape=input_shape,
        kernel_regularizer=L1L2(l1=1e-5, l2=1e-4)
    ))
    model.add(Dropout(0.2))
    
    for i in range(1, n_layers):
        return_seq = (i != n_layers - 1)
        model.add(LSTM(
            neurons // (2 ** i),  
            return_sequences=return_seq,
            kernel_regularizer=L1L2(l1=1e-5, l2=1e-4)
        ))
        model.add(Dropout(0.2))
    
    model.add(Dense(25, activation='relu'))
    model.add(Dense(1))
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss=mape_loss, optimizer=optimizer)
    return model

def gerar_previsoes(caminho_arquivo):
    # Carregar e preparar dados
    df = pd.read_csv(caminho_arquivo, sep=';')
    df['AM_REFERENCIA'] = pd.to_datetime(df['AM_REFERENCIA'], format='%Y%m')
    df_aggregated = df.groupby('AM_REFERENCIA')['HCLQTCON'].sum().reset_index()
    df_aggregated = df_aggregated.sort_values(by='AM_REFERENCIA')
    
    # Filtrar período de treino (2015-2024)
    train_data = df_aggregated[(df_aggregated['AM_REFERENCIA'] >= '2015-01-01') & 
                              (df_aggregated['AM_REFERENCIA'] <= '2024-12-31')]
    
    # Normalização
    scaler = MinMaxScaler(feature_range=(0, 1))
    ts_scaled = scaler.fit_transform(train_data[['HCLQTCON']]).flatten()
    
    # Criar datasets de treino
    look_back = 12  # Usando 12 meses como lookback para capturar sazonalidade anual
    X_train, Y_train = create_dataset(ts_scaled, look_back)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    
    # Construir e treinar o modelo final
    model = build_dynamic_model(
        input_shape=(X_train.shape[1], 1),
        neurons=128,
        learning_rate=0.001,
        n_layers=2
    )
    
    early_stop = EarlyStopping(monitor='loss', patience=20)
    model.fit(
        X_train, Y_train,
        epochs=200,
        batch_size=4,
        verbose=1,
        callbacks=[early_stop]
    )
    
    # Gerar previsões para 2025-2035
    forecast_periods = 132  # 11 anos (jan/2025 a jan/2035)
    forecast = []
    current_batch = ts_scaled[-look_back:].reshape(1, look_back, 1)
    
    for i in range(forecast_periods):
        current_pred = model.predict(current_batch)[0]
        forecast.append(current_pred)
        current_batch = np.append(current_batch[:,1:,:], [[current_pred]], axis=1)
    
    forecast = scaler.inverse_transform(np.array(forecast).reshape(-1, 1)).flatten()
    
    # Criar datas para o período de previsão
    last_date = train_data['AM_REFERENCIA'].iloc[-1]
    forecast_dates = pd.date_range(
        start=last_date + pd.DateOffset(months=1),
        periods=forecast_periods,
        freq='MS'
    )
    
    # Criar DataFrame com as previsões
    forecast_df = pd.DataFrame({
        'Data': forecast_dates,
        'Valor Previsto': forecast
    })
    
    # Salvar em CSV
    forecast_df.to_csv('previsao_consumo_2025_2035.csv', index=False)
    print("Previsões salvas em 'previsao_consumo_2025_2035.csv'")
    
    # Plotar resultados
    plt.figure(figsize=(15, 7))
    plt.plot(train_data['AM_REFERENCIA'], train_data['HCLQTCON'], label='Dados Históricos')
    plt.plot(forecast_df['Data'], forecast_df['Valor Previsto'], label='Previsão 2025-2035', color='red')
    plt.title('Previsão de Consumo Mensal (2025-2035)')
    plt.xlabel('Data')
    plt.ylabel('Consumo')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
    return forecast_df

# Execução
if __name__ == "__main__":
    path = 'E:\\Projetos\\ABMS-WP'
    caminho_arquivo = os.path.join(path, 'includes\\Tabela_consumo_Itapua_120m.csv')
    previsoes = gerar_previsoes(caminho_arquivo)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject